## Лабораторная работа №1

#### Задание 1
Найти набор данных (датасет) для классификации удовлетворяющий следующим условиям: более 10 000 строк, более 20 столбцов, разные типы в столбцах, обязательно наличие целевого признака (таргета).

#### Задание 2
Провести классификацию найденного датасета, методом к- ближайших соседей. В формате Markdown писать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

#### Ход работы:

Если не установлен xlrd (для работы с excel)

In [1]:
!pip install xlrd

     ---------------------------------------- 0.0/96.5 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/96.5 kB ? eta -:--:--
     ------------ ------------------------- 30.7/96.5 kB 445.2 kB/s eta 0:00:01
     ------------------------ ------------- 61.4/96.5 kB 550.5 kB/s eta 0:00:01
     -------------------------------------- 96.5/96.5 kB 690.3 kB/s eta 0:00:00


Импортируем библиотеки и загружаем датасет

In [6]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

file_path = "../Dataset.xls"
df = pd.read_excel(file_path)

df

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,220000,1,3,1,39,0,0,0,0,...,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29996,29997,150000,1,3,2,43,-1,-1,-1,-1,...,8979,5190,0,1837,3526,8998,129,0,0,0
29997,29998,30000,1,2,2,37,4,3,2,-1,...,20878,20582,19357,0,0,22000,4200,2000,3100,1
29998,29999,80000,1,3,1,41,1,-1,0,0,...,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


Разделение данных и нормализация

In [7]:
# Разделение на признаки (X) и целевую переменную (y)
X = df.drop('default payment next month', axis=1)
y = df['default payment next month']

# Разделение данных на тренировочный и тестовый наборы
# random_state=42 - гарантирует, что данные каждый раз будут одинакого разбиваться
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабирование признаков (нормализация)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Подбор гиперпараметров с помощью кросс-валидации и определение наилучшего k

In [21]:
param_grid = {'n_neighbors': list(range(2, 11)), 'metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski']}

# Создание модели k-ближайших соседей
knn = KNeighborsClassifier()
grid = GridSearchCV(knn, param_grid, refit=True, verbose=3, cv=5)
# Обучение модели
grid.fit(X_train_scaled, y_train)

best_params = grid.best_params_
best_knn = grid.best_estimator_

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV 1/5] END ...metric=euclidean, n_neighbors=2;, score=0.784 total time=   0.3s
[CV 2/5] END ...metric=euclidean, n_neighbors=2;, score=0.792 total time=   0.3s
[CV 3/5] END ...metric=euclidean, n_neighbors=2;, score=0.787 total time=   0.4s
[CV 4/5] END ...metric=euclidean, n_neighbors=2;, score=0.788 total time=   0.3s
[CV 5/5] END ...metric=euclidean, n_neighbors=2;, score=0.779 total time=   0.3s
[CV 1/5] END ...metric=euclidean, n_neighbors=3;, score=0.778 total time=   0.3s
[CV 2/5] END ...metric=euclidean, n_neighbors=3;, score=0.787 total time=   0.3s
[CV 3/5] END ...metric=euclidean, n_neighbors=3;, score=0.775 total time=   0.3s
[CV 4/5] END ...metric=euclidean, n_neighbors=3;, score=0.774 total time=   0.3s
[CV 5/5] END ...metric=euclidean, n_neighbors=3;, score=0.765 total time=   0.3s
[CV 1/5] END ...metric=euclidean, n_neighbors=4;, score=0.796 total time=   0.3s
[CV 2/5] END ...metric=euclidean, n_neighbors=4

[CV 2/5] END ...metric=chebyshev, n_neighbors=4;, score=0.802 total time=   1.5s
[CV 3/5] END ...metric=chebyshev, n_neighbors=4;, score=0.790 total time=   1.5s
[CV 4/5] END ...metric=chebyshev, n_neighbors=4;, score=0.796 total time=   1.5s
[CV 5/5] END ...metric=chebyshev, n_neighbors=4;, score=0.790 total time=   1.7s
[CV 1/5] END ...metric=chebyshev, n_neighbors=5;, score=0.788 total time=   1.5s
[CV 2/5] END ...metric=chebyshev, n_neighbors=5;, score=0.801 total time=   1.5s
[CV 3/5] END ...metric=chebyshev, n_neighbors=5;, score=0.794 total time=   1.4s
[CV 4/5] END ...metric=chebyshev, n_neighbors=5;, score=0.795 total time=   1.4s
[CV 5/5] END ...metric=chebyshev, n_neighbors=5;, score=0.791 total time=   1.4s
[CV 1/5] END ...metric=chebyshev, n_neighbors=6;, score=0.796 total time=   1.5s
[CV 2/5] END ...metric=chebyshev, n_neighbors=6;, score=0.807 total time=   1.5s
[CV 3/5] END ...metric=chebyshev, n_neighbors=6;, score=0.796 total time=   1.5s
[CV 4/5] END ...metric=cheby

Проверка на тестовых данных

In [22]:
# Предсказания на тестовой выборке с использованием лучшей модели
predictions = best_knn.predict(X_test_scaled)

Вывод результатов:

In [23]:
# Оценка точности лучшей модели
accuracy = accuracy_score(y_test, predictions)
print(f"\nЛучшие параметры: {best_params}")
print(f"Точность: {accuracy}")

# Отчет по классификации
print("\nОтчет по классификации:")
print(classification_report(y_test, predictions))


Лучшие параметры: {'metric': 'euclidean', 'n_neighbors': 10}
Точность: 0.8078333333333333

Отчет по классификации:
              precision    recall  f1-score   support

           0       0.83      0.95      0.89      4687
           1       0.64      0.29      0.39      1313

    accuracy                           0.81      6000
   macro avg       0.73      0.62      0.64      6000
weighted avg       0.78      0.81      0.78      6000

